# Chess Engine Training on Google Colab

This notebook will help you train the chess engine using Google Colab's GPU. Follow these steps:

1. Mount your Google Drive to save model checkpoints
2. Install required packages
3. Set up the project structure
4. Train the model
5. Download the trained model

In [25]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install python-chess torch numpy tqdm matplotlib

# Create project structure
!mkdir -p models

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ChessNet(nn.Module):
    def __init__(self):
        super(ChessNet, self).__init__()
        self.conv1 = nn.Conv2d(13, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(256 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc_value = nn.Linear(512, 1)
        self.fc_policy = nn.Linear(512, 4672)  # All possible moves

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        x = x.view(-1, 256 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        value = torch.tanh(self.fc_value(x))
        policy = F.log_softmax(self.fc_policy(x), dim=1)

        return value, policy

def load_model(path):
    model = ChessNet()
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

In [41]:
import torch
import torch.nn.functional as F
import numpy as np
import chess

class ChessBoard:
    def __init__(self):
        self.board = chess.Board()
        self.piece_values = {
            chess.PAWN: 1,
            chess.KNIGHT: 3,
            chess.BISHOP: 3,
            chess.ROOK: 5,
            chess.QUEEN: 9,
            chess.KING: 0
        }

    def get_board_state(self):
        """Convert board to neural network input format"""
        state = np.zeros((13, 8, 8), dtype=np.float32)
        for square in chess.SQUARES:
            piece = self.board.piece_at(square)
            if piece is None:
                state[12][square // 8][square % 8] = 1
            else:
                piece_idx = piece.piece_type - 1 + (6 if piece.color else 0)
                state[piece_idx][square // 8][square % 8] = 1
        return state

    def get_legal_moves(self):
        return [move.uci() for move in self.board.legal_moves]

    def make_move(self, move_uci):
        try:
            move = chess.Move.from_uci(move_uci)
            if move in self.board.legal_moves:
                self.board.push(move)
                return True
            return False
        except ValueError:
            return False

    def is_game_over(self):
        return self.board.is_game_over()

    def get_result(self):
        if not self.is_game_over():
            return 0.0  # Return 0.0 for ongoing games
        result = self.board.result()
        if result == "1-0":
            return 1.0
        elif result == "0-1":
            return -1.0
        else:
            return 0.0  # Return 0.0 for draws

class SelfPlayTrainer:
    def __init__(self, model_path=None):
        self.model = ChessNet()
        if model_path:
            self.model.load_state_dict(torch.load(model_path))
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.value_criterion = nn.MSELoss()
        self.policy_criterion = nn.CrossEntropyLoss()

    def self_play_game(self, temperature=1.0):
        board = ChessBoard()
        agent = ChessAgent(self.model)
        states = []
        moves = []

        while not board.is_game_over():
            state = board.get_board_state()
            move = agent.select_move(temperature)

            if move is None:  # No legal moves available
                break

            if board.make_move(move):  # Only append if move was successful
                states.append(state)
                moves.append(agent._move_to_index(move))  # Convert move to index
            else:
                break  # Stop if move was invalid

        result = board.get_result()
        return states, moves, result

    def train_step(self, states, moves, result):
        if not states or not moves:  # Skip if no valid moves were made
            return 0.0

        self.model.train()
        self.optimizer.zero_grad()

        states = np.array(states)
        states = torch.FloatTensor(states)
        moves = torch.LongTensor(moves)  # Now moves are already indices
        result = torch.FloatTensor([result])

        value, policy = self.model(states)
        value_loss = self.value_criterion(value.squeeze(), result)
        policy_loss = self.policy_criterion(policy, moves)
        loss = value_loss + policy_loss

        loss.backward()
        self.optimizer.step()
        return loss.item()

    def train(self, num_games=100, num_epochs=10):
        for game in tqdm(range(num_games)):
            states, moves, result = self.self_play_game()
            if states and moves:  # Only train if we have valid data
                for epoch in range(num_epochs):
                    loss = self.train_step(states, moves, result)
                if (game + 1) % 10 == 0:
                    torch.save(self.model.state_dict(), f'models/checkpoint_{game+1}.pt')

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np

class SelfPlayTrainer:
    def __init__(self, model_path=None):
        self.model = ChessNet()
        if model_path:
            self.model.load_state_dict(torch.load(model_path))
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.value_criterion = nn.MSELoss()
        self.policy_criterion = nn.CrossEntropyLoss()

    def self_play_game(self, temperature=1.0):
        board = ChessBoard()
        agent = ChessAgent(self.model)
        states = []
        moves = []

        while not board.is_game_over():
            state = board.get_board_state()
            move = agent.select_move(temperature)

            if move is None:  # No legal moves available
                break

            if board.make_move(move):  # Only append if move was successful
                states.append(state)
                moves.append(agent._move_to_index(move))  # Convert move to index
            else:
                break  # Stop if move was invalid

        result = board.get_result()
        return states, moves, result

    def train_step(self, states, moves, result):
        if not states or not moves:  # Skip if no valid moves were made
            return 0.0

        self.model.train()
        self.optimizer.zero_grad()

        states = np.array(states)
        states = torch.FloatTensor(states)
        moves = torch.LongTensor(moves)  # Now moves are already indices
        result = torch.FloatTensor([result])

        value, policy = self.model(states)
        value_loss = self.value_criterion(value.squeeze(), result)
        policy_loss = self.policy_criterion(policy, moves)
        loss = value_loss + policy_loss

        loss.backward()
        self.optimizer.step()
        return loss.item()

    def train(self, num_games=100, num_epochs=10):
        for game in tqdm(range(num_games)):
            states, moves, result = self.self_play_game()
            if states and moves:  # Only train if we have valid data
                for epoch in range(num_epochs):
                    loss = self.train_step(states, moves, result)
                if (game + 1) % 10 == 0:
                    torch.save(self.model.state_dict(), f'models/checkpoint_{game+1}.pt')

In [43]:
class ChessCLI:
    def __init__(self, model_path):
        self.model = load_model(model_path)
        self.agent = ChessAgent(self.model)
        self.board = ChessBoard()

    def print_board(self):
        print(self.board.board)

    def play(self, human_plays_white=True):
        while not self.board.is_game_over():
            self.print_board()
            if (human_plays_white and self.board.board.turn) or \
               (not human_plays_white and not self.board.board.turn):
                move = input("Enter your move (UCI format, e.g., 'e2e4'): ")
                try:
                    self.board.make_move(move)
                except ValueError:
                    print("Invalid move! Try again.")
                    continue
            else:
                move = self.agent.select_move()
                print(f"Engine plays: {move}")
                self.board.make_move(move)

        self.print_board()
        result = self.board.get_result()
        if result == 1:
            print("White wins!")
        elif result == -1:
            print("Black wins!")
        else:
            print("Draw!")

In [44]:
# Training parameters
NUM_GAMES = 10
NUM_EPOCHS = 1
MODEL_SAVE_PATH = '/content/drive/MyDrive/chess_engine/models'

# Create trainer
trainer = SelfPlayTrainer()

# Train the model
print("Starting training...")
trainer.train(num_games=NUM_GAMES, num_epochs=NUM_EPOCHS)
print("Training completed!")

Starting training...


  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 10/10 [00:03<00:00,  2.85it/s]

Training completed!


In [45]:
# Download the trained model
!cp {MODEL_SAVE_PATH}/checkpoint_{NUM_GAMES}.pt /content/models/checkpoint_{NUM_GAMES}.pt
!zip -r /content/chess_model.zip /content/models/

from google.colab import files
files.download('/content/chess_model.zip')

cp: cannot stat '/content/drive/MyDrive/chess_engine/models/checkpoint_10.pt': No such file or directory
  adding: content/models/ (stored 0%)
  adding: content/models/checkpoint_10.pt (deflated 9%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>